# Q2

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch({'host': 'localhost', 'port': 9200, 'scheme': 'http'})


In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "dt_faq"

es.indices.create(index=index_name, body=index_settings)

C:\Users\m.salama\AppData\Local\Temp\ipykernel_19676\2000385392.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=index_settings)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [dt_faq/I3-z7RU0RFaxiABdDOGnvg] already exists')

In [11]:
from tqdm import tqdm

for doc in tqdm(documents):
    es.index(index='dt_faq', document=doc)

100%|██████████| 948/948 [01:03<00:00, 15.04it/s]


In [5]:
from pprint import pprint

def search(q, topk=5, index_name='dt_faq', filters=None):
    search_query = {
            "size": topk,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": q,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    "filter": filters
                }
            }
        }

    res = es.search(index=index_name, body=search_query)

    results = []
    for hit in res['hits']['hits']:
        results.append(hit["_source"])
        pprint(hit['_score'])
        pprint(hit["_source"])
        print('---')
        print() 
    
    return results

In [7]:
q =  "How do execute a command on a Kubernetes pod?"
q ="How do copy a file to a Docker container?"
filters = {
    "term": {
        "course": "machine-learning-zoomcamp"
    }
}
contx = search(q, filters=filters, topk=3)

73.38676
{'course': 'machine-learning-zoomcamp',
 'question': 'How do I debug a docker container?',
 'section': '5. Deploying Machine Learning Models',
 'text': 'Launch the container image in interactive mode and overriding the '
         'entrypoint, so that it starts a bash command.\n'
         'docker run -it --entrypoint bash <image>\n'
         'If the container is already running, execute a command in the '
         'specific container:\n'
         'docker ps (find the container-id)\n'
         'docker exec -it <container-id> bash\n'
         '(Marcos MJD)'}
---

66.688705
{'course': 'machine-learning-zoomcamp',
 'question': 'How do I copy files from my local machine to docker container?',
 'section': '5. Deploying Machine Learning Models',
 'text': 'You can copy files from your local machine into a Docker container '
         "using the docker cp command. Here's how to do it:\n"
         'To copy a file or directory from your local machine into a running '
         'Docker conta

C:\Users\m.salama\AppData\Local\Temp\ipykernel_19676\1936798363.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=index_name, body=search_query)


In [14]:
context_template = """
Q: {question}
A: {text}
""".strip()

query = """"
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}
CONTEXT:
{context}
""".strip()

In [15]:
all_contx = []
for c in contx:
    all_contx.append(context_template.format(question=c['question'], text=c['text']))

query = query.format(question=q, context='\n\n'.join(all_contx)).strip()
print(query)

"
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?
CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contai

In [16]:
len(query)

1447

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [20]:
print(len(encoding.encode(query)))

321
